## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [20]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="bitcoin"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('bitcoin_train_9_1.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,9.4545,10.3018,9.4297,10.1893,796676,USD
1,2020-06-02,10.1907,10.2073,9.3473,9.5276,1058556,USD
2,2020-06-03,9.5270,9.6672,9.4217,9.6672,558116,USD
3,2020-06-04,9.6672,9.8644,9.4909,9.7944,593664,USD
4,2020-06-05,9.7940,9.8461,9.6287,9.6312,527054,USD
...,...,...,...,...,...,...,...
653,2022-03-16,39.2825,41.7016,38.9532,41.1187,4468696576,USD
654,2022-03-17,41.1187,41.4060,40.5578,40.9148,1897994368,USD
655,2022-03-18,40.9146,42.3084,40.2348,41.7678,2314979328,USD
656,2022-03-19,41.7680,42.3860,41.5296,42.2330,224590832,USD


Chuẩn hóa dữ liệu

In [21]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Close'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Close'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Close'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Close'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Close'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}_9_1_standardized.csv")
print(df)

           Date     Open     High      Low    Close      Volume Currency  \
20   2020-06-21   9.3588   9.4112   9.2885   9.2964      335994      USD   
21   2020-06-22   9.2965   9.7519   9.2858   9.6837      523319      USD   
22   2020-06-23   9.6839   9.7156   9.5818   9.6246      375494      USD   
23   2020-06-24   9.6246   9.6674   9.2230   9.3020      515777      USD   
24   2020-06-25   9.3008   9.3273   9.0226   9.2475      524453      USD   
..          ...      ...      ...      ...      ...         ...      ...   
653  2022-03-16  39.2825  41.7016  38.9532  41.1187  4468696576      USD   
654  2022-03-17  41.1187  41.4060  40.5578  40.9148  1897994368      USD   
655  2022-03-18  40.9146  42.3084  40.2348  41.7678  2314979328      USD   
656  2022-03-19  41.7680  42.3860  41.5296  42.2330   224590832      USD   
657  2022-03-20  42.2410  42.3010  40.9220  41.2760   308448032      USD   

        H-L     O-C      SMA_7     SMA_14     SMA_21      SD_7     SD_21  
20   0.1227 

Scale Miền giá trị lại

In [22]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Close']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))

In [23]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

In [24]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}91.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 9s 29ms/step - loss: 0.0315
Epoch 2/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0102
Epoch 3/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0154
Epoch 4/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0086
Epoch 5/60
40/40 [==============================] - 1s 29ms/step - loss: 0.0062
Epoch 6/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0056
Epoch 7/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0056
Epoch 8/60
40/40 [==============================] - 1s 29ms/step - loss: 0.0054
Epoch 9/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0041
Epoch 10/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0045
Epoch 11/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0041
Epoch 12/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0039
Epoch 13/60
40/40 [==================